# Portfolio Optimization

Portfolio Optimization helps risk-averse investors construct portfolios to maximize expected return given a certain level of market risk, emphasizing the inherent relationship between risk and reward. We will begin by running an example of the Monte Carlo Simulation for an optimal portfolio with resulting returns. Then we will create an Efficient Frontier which is used to identify a set of optimal portfolios that offers the highest expected return for a defined level of risk or the lowest risk for a given level of expected return. This will be done using hvPlot and HoloViz which will allow users to visualize the trade-offs between risk and return. Lastly, we will combine all our analysis into a Panel app that enables users to dynamically explore the Efficient Frontier, adjust parameters, and visualize the resulting portfolios, streamlining the portfolio optimization process.

We will be using a variety of HoloViz tools to support portfolio optimization analysis:
- [hvPlot](https://hvplot.holoviz.org/) - generates visualizations such as line graphs and histograms from Pandas' data
- [HoloViews](https://holoviews.org/) - creates the efficent frontier curve after the portfolio is optimized
- [Panel](https://panel.holoviz.org/) - combines all the visualizations and graphs into a single interactive app
  
![image.png](dashboard.png)

## Monte Carlo Simulation for Optimization Search


Monte Carlo simulations are used by analyst to determine the expected value and optimal distribution of a portfolio. 

In [202]:
from io import BytesIO
import numpy as np
import pandas as pd
import hvplot.pandas  # noqa
import panel as pn

In [203]:
stocks = pd.read_csv('./data/stocks.csv', index_col='Date', parse_dates=True)
stocks.head()

aapl      cisco         ibm    amzn
Date                                                
2012-01-03  53.063218  15.752778  160.830881  179.03
2012-01-04  53.348386  16.057180  160.174781  177.51
2012-01-05  53.940658  15.997991  159.415086  177.61
2012-01-06  54.504543  15.938801  157.584912  182.61
2012-01-09  54.418089  16.040268  156.764786  178.56

In [204]:
mean_daily_ret = stocks.pct_change(1).mean()
mean_daily_ret

aapl     0.000750
cisco    0.000599
ibm      0.000081
amzn     0.001328
dtype: float64

In [205]:
stocks.pct_change(1).corr()

aapl     cisco       ibm      amzn
aapl   1.000000  0.301990  0.297498  0.235487
cisco  0.301990  1.000000  0.424672  0.284470
ibm    0.297498  0.424672  1.000000  0.258492
amzn   0.235487  0.284470  0.258492  1.000000

# Simulating Thousands of Possible Allocations

In [206]:
stocks.head()

aapl      cisco         ibm    amzn
Date                                                
2012-01-03  53.063218  15.752778  160.830881  179.03
2012-01-04  53.348386  16.057180  160.174781  177.51
2012-01-05  53.940658  15.997991  159.415086  177.61
2012-01-06  54.504543  15.938801  157.584912  182.61
2012-01-09  54.418089  16.040268  156.764786  178.56

In [207]:
stock_normed = stocks/stocks.iloc[0]
timeseries = stock_normed.hvplot()
timeseries

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [208]:
stock_daily_ret = stocks.pct_change(1)
stock_daily_ret.head()

aapl     cisco       ibm      amzn
Date                                              
2012-01-03       NaN       NaN       NaN       NaN
2012-01-04  0.005374  0.019324 -0.004079 -0.008490
2012-01-05  0.011102 -0.003686 -0.004743  0.000563
2012-01-06  0.010454 -0.003700 -0.011481  0.028152
2012-01-09 -0.001586  0.006366 -0.005204 -0.022178

## Log Returns vs Arithmetic Returns

We will now switch over to using log returns instead of arithmetic returns, for many of our use cases they are almost the same,but most technical analyses require detrending/normalizing the time series and using log returns is a nice way to do that.
Log returns are convenient to work with in many of the algorithms we will encounter.

For a full analysis of why we use log returns, check [this great article](https://quantivity.wordpress.com/2011/02/21/why-log-returns/).


In [209]:
log_ret = np.log(stocks/stocks.shift(1))
log_ret.head()

aapl     cisco       ibm      amzn
Date                                              
2012-01-03       NaN       NaN       NaN       NaN
2012-01-04  0.005360  0.019139 -0.004088 -0.008526
2012-01-05  0.011041 -0.003693 -0.004754  0.000563
2012-01-06  0.010400 -0.003707 -0.011547  0.027763
2012-01-09 -0.001587  0.006346 -0.005218 -0.022428

In [210]:
log_ret_hists = log_ret.hvplot.hist(bins=100, subplots=True, width=400, \
    group_label='Ticker', grid=True).cols(2)
log_ret_hists

:NdLayout   [Default]
   :Histogram   [amzn]   (amzn_count)

In [211]:
log_ret.describe().transpose()

count      mean       std       min       25%       50%       75%  \
aapl   1257.0  0.000614  0.016466 -0.131875 -0.007358  0.000455  0.009724   
cisco  1257.0  0.000497  0.014279 -0.116091 -0.006240  0.000213  0.007634   
ibm    1257.0  0.000011  0.011819 -0.086419 -0.005873  0.000049  0.006477   
amzn   1257.0  0.001139  0.019362 -0.116503 -0.008534  0.000563  0.011407   

            max  
aapl   0.085022  
cisco  0.118862  
ibm    0.049130  
amzn   0.146225

We will be multiplying the `log_ret.mean()` by 252 because there 252 trading days in a year. 

In [212]:
log_ret.mean() * 252

aapl     0.154803
cisco    0.125291
ibm      0.002788
amzn     0.287153
dtype: float64

In [213]:
# Compute pairwise covariance of columns
log_ret.cov()

aapl     cisco       ibm      amzn
aapl   0.000271  0.000071  0.000057  0.000075
cisco  0.000071  0.000204  0.000072  0.000079
ibm    0.000057  0.000072  0.000140  0.000059
amzn   0.000075  0.000079  0.000059  0.000375

In [214]:
log_ret.cov()*252

aapl     cisco       ibm      amzn
aapl   0.068326  0.017854  0.014464  0.018986
cisco  0.017854  0.051381  0.018029  0.019956
ibm    0.014464  0.018029  0.035203  0.014939
amzn   0.018986  0.019956  0.014939  0.094470

## Single Run for Some Random Allocation

In [215]:
# Set seed (optional)
np.random.seed(101)

# Stock Columns
print('Stocks')
print(stocks.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(4))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')

# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(log_ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)


Stocks
Index(['aapl', 'cisco', 'ibm', 'amzn'], dtype='object')


Creating Random Weights
[0.51639863 0.57066759 0.02847423 0.17152166]


Rebalance to sum to 1.0
[0.40122278 0.44338777 0.02212343 0.13326603]


Expected Portfolio Return
0.15599272049632015


Expected Volatility
0.18502649565909485


Sharpe Ratio
0.843083148392604


Great! Now we can just run this many times over!

In [216]:
num_ports = 15000

all_weights = np.zeros((num_ports,len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(4))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [217]:
sharpe_arr.max()

1.0303260551271078

In [218]:
sharpe_arr.argmax()

1419

In [219]:
all_weights[1419,:]

array([0.26188068, 0.20759516, 0.00110226, 0.5294219 ])

In [220]:
max_sr_ret = ret_arr[1419]
max_sr_vol = vol_arr[1419]

## Plotting the data

In [221]:
import holoviews as hv

In [222]:
scatter = hv.Scatter((vol_arr, ret_arr, sharpe_arr), 'Volatility', ['Return', 'Sharpe Ratio'])
max_sharpe = hv.Scatter([(max_sr_vol,max_sr_ret)])

scatter.opts(color='Sharpe Ratio', cmap='plasma', width=600, height=400, colorbar=True, padding=0.1) *\
max_sharpe.opts(color='red', line_color='black', size=10)

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)

# Mathematical Optimization

There are much better ways to find good allocation weights than just guess and check! We can use optimization functions to find the ideal weights mathematically!

### Functionalize Return and SR operations

In [223]:
def get_ret_vol_sr(weights):
    """
    Takes in weights, returns array or return,volatility, sharpe ratio
    """
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [224]:
from scipy.optimize import minimize

To fully understand all the parameters, check out the [`scipy.optimize.minimize` documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).

In [225]:
#help(minimize)

Optimization works as a minimization function, since we actually want to maximize the Sharpe Ratio, we will need to turn it negative so we can minimize the negative sharpe (same as maximizing the postive sharpe)

In [226]:
def neg_sharpe(weights):
    return  get_ret_vol_sr(weights)[2] * -1

In [227]:
# Contraints
def check_sum(weights):
    '''
    Returns 0 if sum of weights is 1.0
    '''
    return np.sum(weights) - 1

In [228]:
# By convention of minimize function it should be a function that returns zero for conditions
cons = ({'type':'eq','fun': check_sum})

In [229]:
# 0-1 bounds for each weight
bounds = ((0, 1), (0, 1), (0, 1), (0, 1))

In [230]:
# Initial Guess (equal distribution)
init_guess = [0.25,0.25,0.25,0.25]

In [231]:
# Sequential Least SQuares Programming (SLSQP).
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [232]:
opt_results

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.0307168703355527
       x: [ 2.663e-01  2.042e-01  1.098e-17  5.295e-01]
     nit: 7
     jac: [ 5.643e-05  4.183e-05  3.399e-01 -4.449e-05]
    nfev: 35
    njev: 7

In [233]:
opt_results.x

array([2.66289782e-01, 2.04189807e-01, 1.09775470e-17, 5.29520411e-01])

In [234]:
get_ret_vol_sr(opt_results.x)

array([0.21885916, 0.21233683, 1.03071687])

# All Optimal Portfolios (Efficient Frontier)

The [efficient frontier](https://www.investopedia.com/terms/e/efficientfrontier.asp) is the set of optimal portfolios that offers the highest expected return for a defined level of risk or the lowest risk for a given level of expected return. Portfolios that lie below the efficient frontier are sub-optimal, because they do not provide enough return for the level of risk. Portfolios that cluster to the right of the efficient frontier are also sub-optimal, because they have a higher level of risk for the defined rate of return.

In [235]:
# Our returns go from 0 to somewhere along 0.3
# Create a linspace number of points to calculate x on
frontier_y = np.linspace(0,0.3,100) # Change 100 to a lower number for slower computers!

In [236]:
def minimize_volatility(weights):
    return  get_ret_vol_sr(weights)[1] 

In [237]:
frontier_volatility = []

for possible_return in frontier_y:
    # function for return
    cons = ({'type':'eq','fun': check_sum},
            {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
    
    result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])

In [238]:
ef_graph = scatter * hv.Curve((frontier_volatility, frontier_y)).opts(color='green', line_dash='dashed')
ef_graph

:Overlay
   .Scatter.I :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Curve.I   :Curve   [x]   (y)

Now that we have a graph of the efficient frontier, let's take this a step further by building a Panel app around it. The app will allow a user to upload their own CSV of stock time series which will be analyzed to create a variety of graphs including the efficient frontier. We'll begin by first creating the sidebar for user input by using the `FileInput`, `IntSlider`, and `Button` widgets.

In [239]:
pn.extension('tabulator', design='material', template='material', theme_toggle=True, loading_indicator=True)
file_input = pn.widgets.FileInput(sizing_mode='stretch_width')
selector = pn.widgets.MultiSelect(
    name='Select stocks', sizing_mode='stretch_width',
    options=stocks.columns.to_list()
)


n_samples = pn.widgets.IntSlider(
    name='Random samples', value=10_000, start=1000, end=20_000, step=1000, sizing_mode='stretch_width'
)
button = pn.widgets.Button(name='Run Analysis', sizing_mode='stretch_width')
posxy = hv.streams.Tap(x=None, y=None)

text = """
#  Portfolio optimization

This application performs portfolio optimization given a set of stock time series.

To optimize your portfolio:

1. Upload a CSV of the daily stock time series for the stocks you are considering
2. Select the stocks to be included.
3. Run the Analysis
4. Click on the Return/Volatility plot to select the desired risk/reward profile

Upload a CSV containing stock data:
"""

explanation = """
The code for this app was taken from [this excellent introduction to Python for Finance](https://github.com/PrateekKumarSingh/Python/tree/master/Python%20for%20Finance/Python-for-Finance-Repo-master).
To learn some of the background and theory about portfolio optimization see [this notebook](https://github.com/PrateekKumarSingh/Python/blob/master/Python%20for%20Finance/Python-for-Finance-Repo-master/09-Python-Finance-Fundamentals/02-Portfolio-Optimization.ipynb).
"""

sidebar = pn.layout.WidgetBox(
    pn.pane.Markdown(text, margin=(0, 10)),
    file_input,
    selector,
    n_samples,
    explanation,
    max_width=350,
    sizing_mode='stretch_width'
).servable(area='sidebar')

sidebar

WidgetBox(design=<class 'panel.theme.materi..., max_width=350, sizing_mode='stretch_width')
    [0] Markdown(str, design=<class 'panel.theme.materi..., margin=(0, 10))
    [1] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [2] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=['aapl', 'cisco', ...], sizing_mode='stretch_width')
    [3] IntSlider(design=<class 'panel.theme.materi..., end=20000, name='Random samples', sizing_mode='stretch_width', start=1000, step=1000, value=10000)
    [4] Markdown(str, design=<class 'panel.theme.materi...)

Now that we have created the sidebar component, we will create a function to read the inputted file. We will also use the `MultiSelect` widget to allow the user to select only certain stocks to be included in the portfolio

In [240]:
@pn.cache
def get_stocks(data):
    if data is None:
        stock_file = 'https://datasets.holoviz.org/stocks/v1/stocks.csv'
    else:
        stock_file = BytesIO(data)
    return pd.read_csv(stock_file, index_col='Date', parse_dates=True)


file_input = pn.widgets.FileInput(sizing_mode='stretch_width')

stocks = hvplot.bind(get_stocks, file_input).interactive()
stocks = pn.rx(get_stocks)(file_input)

selector = pn.widgets.MultiSelect(
    name='Select stocks', sizing_mode='stretch_width',
    options=stocks.columns.to_list()
)

selected_stocks = stocks.rx.pipe(
    lambda df, cols: df[cols] if cols else df, selector
)

In [241]:
sidebar

WidgetBox(design=<class 'panel.theme.materi..., max_width=350, sizing_mode='stretch_width')
    [0] Markdown(str, design=<class 'panel.theme.materi..., margin=(0, 10))
    [1] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [2] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=['aapl', 'cisco', ...], sizing_mode='stretch_width')
    [3] IntSlider(design=<class 'panel.theme.materi..., end=20000, name='Random samples', sizing_mode='stretch_width', start=1000, step=1000, value=10000)
    [4] Markdown(str, design=<class 'panel.theme.materi...)

Then we will define a variety of analytical functions to help us find the optimal portfolio allocation point on the efficient frontier visualization. The compute_frontier() function computes the efficient frontier using numerical optimization to find the minimum volatility portfolio for a range of expected returns. The find_best_allocation() function finds the optimal portfolio allocation that achieves the desired volatility and return.

In [242]:
def check_sum(weights):
    return np.sum(weights) - 1

def get_return(mean_ret, weights):
    return np.sum(mean_ret * weights) * 252

def get_volatility(log_ret, weights):
    return np.sqrt(np.dot(weights.T, np.dot(np.cov(log_ret[1:], rowvar=False) * 252, weights)))

def minimize_difference(weights, des_vol, des_ret, log_ret, mean_ret):
    ret = get_return(mean_ret, weights)
    vol = get_volatility(log_ret, weights)
    return abs(des_ret-ret) + abs(des_vol-vol)

@pn.cache
def find_best_allocation(log_return, vol, ret):
    cols = log_return.shape[1]
    vol = vol or 0
    ret = ret or 0
    mean_return = np.nanmean(log_return, axis=0)
    bounds = tuple((0, 1) for i in range(cols))
    init_guess = [1./cols for i in range(cols)]
    cons = (
        {'type':'eq','fun': check_sum},
        {'type':'eq','fun': lambda w: get_return(mean_return, w) - ret},
        {'type':'eq','fun': lambda w: get_volatility(log_return, w) - vol}
    )
    opt = minimize(
        minimize_difference, init_guess, args=(vol, ret, log_return, mean_return),
        bounds=bounds, constraints=cons
    )
    ret = get_return(mean_return, opt.x)
    vol = get_volatility(log_return, opt.x)
    return pd.Series(list(opt.x)+[ret, vol], index=list(log_return.columns)+['Return', 'Volatility'], name='Weight')

Using the functions above, we find our optimal portfolio allocation point and overlay this on our efficient frontier graph. We also use the Tabulator widget to create a summary of the weights of stocks in our optimal portfolio.

In [243]:
# Set up data pipelines

log_return = np.log(selected_stocks/selected_stocks.shift(1))
selected_stocks = stocks
closest_allocation = log_return.pipe(find_best_allocation, posxy.param.x, posxy.param.y)

opts = {'x': 'Volatility', 'y': 'Return', 'responsive': True}

closest_point = closest_allocation.to_frame().T.hvplot.scatter(color='green', line_color='black', size=50, **opts)

ef_graph = ef_graph * closest_point * max_sharpe

summary = pn.pane.Markdown(
    pn.bind(lambda p: f"""
    The selected portfolio has a volatility of {p.Volatility:.2f}, a return of {p.Return:.2f}
    and Sharpe ratio of {p.Return/p.Volatility:.2f}.""", closest_allocation), width=250
)

table = pn.widgets.Tabulator(closest_allocation.to_frame().iloc[:-2])
pn.Row(ef_graph, pn.Column(summary, table), sizing_mode='stretch_both')


Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Row(design=<class 'panel.theme.materi...)
            [0] WidgetBox(design=<class 'panel.theme.materi...)
                [0] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
                [1] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=['aapl', 'cisco', ...], sizing_mode='stretch_width')
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)
    [1] Column(design=<class 'panel.theme.materi...)
        [0] Markdown(str, design=<class 'panel.theme.materi..., width=250)
        [1] Tabulator(design=<class 'panel.theme.materi..., value=         Weight
aapl  ...)

We will also create a line graph that will help us visualize the performance of our portfolio.

In [270]:
investment = pn.widgets.Spinner(name='Investment Value in $', value=5000, step=1000, start=1000, end=100000)
year = pn.widgets.DateRangeSlider(name='Year', value=(stocks.index.rx.value.min(), stocks.index.rx.value.max()), \
                                  start=stocks.index.min(), end=stocks.index.max())

def plot_performance(value_start, value_end, investment=investment.value):
    allocation = closest_allocation.iloc[:-2] * investment
    stocks_between_dates = selected_stocks.loc[value_start:value_end]
    price_on_start_date = selected_stocks.loc[value_start].iloc[0]
    plot = (stocks_between_dates * allocation / price_on_start_date).sum(axis=1).hvplot.line(
        ylabel='Total Value ($)', title='Portfolio performance', responsive=True, min_height=400)
    return plot

performance_plot = pn.bind(plot_performance, year.param.value_start, year.param.value_end, investment)

performance = pn.Column(
    performance_plot,
    sizing_mode='stretch_both'
)

pn.Row(pn.Column(year, investment), performan


Column(design=<class 'panel.theme.materi...)
    [0] DateRangeSlider(design=<class 'panel.theme.materi..., end=Timestamp('2016-12-30 0..., name='Year', start=Timestamp('2012-01-03 0..., value=(Timestamp('2012-01-03 00:..., value_end=Timestamp('2016-12-30 0..., value_start=Timestamp('2012-01-03 0...)
    [1] IntInput(design=<class 'panel.theme.materi..., end=100000, name='Investment Value in $', start=1000, step=1000, value=5000)

We can take advantage of `pn.Tabs` which allows us to put all the graphs we have created into one app very neatly. Wow, we just created an interactive report by just using Holoviews, hvPlot, and Panel! 

In [245]:
main = pn.Tabs(
    ('Analysis', pn.Column(
            pn.Row(
                ef_graph, pn.Column(summary, table),
                sizing_mode='stretch_both'
            ),
            performance,
            sizing_mode='stretch_both'
        )
    ),
    ('Timeseries', timeseries),
    ('Log Return', pn.Column(
        '## Daily normalized log returns',
        'Width of distribution indicates volatility and center of distribution the mean daily return.',
        log_ret_hists,
        sizing_mode='stretch_both'
    )),
    sizing_mode='stretch_both', min_height=1000
)

pn.Row(sidebar, main).servable()

Row(design=<class 'panel.theme.materi...)
    [0] WidgetBox(design=<class 'panel.theme.materi..., max_width=350, sizing_mode='stretch_width')
        [0] Markdown(str, design=<class 'panel.theme.materi..., margin=(0, 10))
        [1] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [2] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=['aapl', 'cisco', ...], sizing_mode='stretch_width')
        [3] IntSlider(design=<class 'panel.theme.materi..., end=20000, name='Random samples', sizing_mode='stretch_width', start=1000, step=1000, value=10000)
        [4] Markdown(str, design=<class 'panel.theme.materi...)
    [1] Tabs(design=<class 'panel.theme.materi..., min_height=1000, sizing_mode='stretch_both')
        [0] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
            [0] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                [0] Row(design=<class 'panel.theme.materi...)
                    [0] Row(design=<class 'panel.theme.materi...)
                        [0] WidgetBox(design=<class 'panel.theme.materi...)
                            [0] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
                            [1] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=['aapl', 'cisco', ...], sizing_mode='stretch_width')
                        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)
                [1] Column(design=<class 'panel.theme.materi...)
                    [0] Markdown(str, design=<class 'panel.theme.materi..., width=250)
                    [1] Tabulator(design=<class 'panel.theme.materi..., value=         Weight
aapl  ...)
            [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                [0] Row(design=<class 'panel.theme.materi...)
                    [0] Row(design=<class 'panel.theme.materi...)
                        [0] WidgetBox(design=<class 'panel.theme.materi...)
                            [0] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
                            [1] DateRangeSlider(design=<class 'panel.theme.materi..., end=Timestamp('2016-12-30 0..., name='Year', start=Timestamp('2012-01-03 0..., value=(Timestamp('2012-01-03 00:..., value_end=Timestamp('2016-12-30 0..., value_start=Timestamp('2012-01-03 0...)
                            [2] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=['aapl', 'cisco', ...], sizing_mode='stretch_width')
                            [3] IntInput(design=<class 'panel.theme.materi..., end=100000, name='Investment Value in $', start=1000, step=1000, value=5000)
                        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)
        [1] HoloViews(NdOverlay, design=<class 'panel.theme.materi..., height=300, name='Timeseries', sizing_mode='fixed', width=700)
        [2] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
            [0] Markdown(str, design=<class 'panel.theme.materi...)
            [1] Markdown(str, design=<class 'panel.theme.materi...)
            [2] HoloViews(NdLayout, design=<class 'panel.theme.materi...)